In [136]:
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 600)
pd.set_option('display.max_colwidth', 300)
import re
from html.parser import HTMLParser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df = pd.read_json('data/ufodata.json', lines=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136494 entries, 0 to 136493
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   _id     136494 non-null  object
 1   url     136494 non-null  object
 2   html    136494 non-null  object
 3   time    136494 non-null  object
dtypes: object(4)
memory usage: 4.2+ MB


In [14]:
df.head(1)['html'].str.contains('McGregor')

0    True
Name: html, dtype: bool

In [15]:
obj = df.loc[0,'html']

In [32]:
pattern = re.compile('<TBODY>.*<\/TBODY>')

In [43]:
re.search(r'<TBODY>.*', obj)

<re.Match object; span=(39621, 39629), match='<TBODY>\r'>

In [44]:
re.findall(r'<TBODY>.*<\/TBODY>', obj)

[]

In [68]:
re.search('</TBODY>', obj).span()[0]

40386

In [28]:
obj[39000:41000]

'"http://www.nuforc.org/webreports/ndxshape.html">by Shape</a> | <a\r\nhref="http://www.nuforc.org/webreports/ndxpost.html">by Posting Date</a></span><span\r\nstyle=\'mso-fareast-font-family:"Times New Roman"\'><o:p></o:p></span></p>\r\n\r\n</div>\r\n\r\n<!--The following token places all object output inside the <BODY> tag.--><TABLE BORDER=1 BGCOLOR=#ffffff CELLSPACING=0><FONT FACE="Calibri" COLOR=#000000><CAPTION><B>National UFO Reporting Center</B></CAPTION></FONT>\r\n\r\n<THEAD>\r\n<TR>\r\n<TH BGCOLOR=#c0c0c0 BORDERCOLOR=#000000 ><FONT style=FONT-SIZE:11pt FACE="Calibri" COLOR=#000000>Sighting Report</FONT></TH>\r\n\r\n</TR>\r\n</THEAD>\r\n<TBODY>\r\n<TR VALIGN=TOP>\r\n<TD BORDERCOLOR=#d0d7e5 ><FONT style=FONT-SIZE:11pt FACE="Calibri" COLOR=#000000>Occurred : 5/6/2017 05:00  (Entered as : 05/06/2017 05:00)<BR>Reported: 5/6/2017 4:10:01 AM 04:10<BR>Posted: 5/6/2017<BR>Location: Camp McGregor, NM<BR>Shape: Light<BR>Duration:10 minutes</FONT></TD>\r\n\r\n</TR>\r\n\r\n<TR VALIGN=TOP>\r

In [31]:
obj[40386]

'<'

In [117]:
def index_finder(row):
    try:
        index = re.search('Occurred : .*', row).span()
        begin, end = index[0], index[1]
        tolist = row[begin:end].replace('\r', '').replace('</FONT></TD>', '').split('<BR>')
        return tolist
    except AttributeError:
        pass

In [113]:
index_finder(obj)

['Occurred : 5/6/2017 05:00  (Entered as : 05/06/2017 05:00)',
 'Reported: 5/6/2017 4:10:01 AM 04:10',
 'Posted: 5/6/2017',
 'Location: Camp McGregor, NM',
 'Shape: Light',
 'Duration:10 minutes']

In [118]:
df['NewCol'] = df['html'].apply(index_finder)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136494 entries, 0 to 136493
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   _id     136494 non-null  object
 1   url     136494 non-null  object
 2   html    136494 non-null  object
 3   time    136494 non-null  object
 4   NewCol  136332 non-null  object
dtypes: object(5)
memory usage: 5.2+ MB


df['NewCol'].tolist()

### Messing around with html.parser

In [178]:
td = soup.find_all('td')

In [183]:
td[1].get_text()

"Light seen over mountain's east of Camp McGregor.  It hovered in one spot.  It looked like a helicopter light at first.  But it was way to bright.  Then it went to a higher height.  Stayed there for a while.  Then disappeared.((NUFORC Note:  Report appears to us to be consistent with the sighting of Venus.  PD))"

In [185]:
text = str(td[0])

In [188]:
text.replace('<td bordercolor="#d0d7e5"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">', '').replace('</font></td>', '').split('<br/>')

['Occurred : 5/6/2017 05:00  (Entered as : 05/06/2017 05:00)',
 'Reported: 5/6/2017 4:10:01 AM 04:10',
 'Posted: 5/6/2017',
 'Location: Camp McGregor, NM',
 'Shape: Light',
 'Duration:10 minutes']

In [250]:
#Approach is to break up the data pulls, first pull text, then pull the table data

tables = []
def parser(row):
    soup = BeautifulSoup(row, 'html.parser')
    try:
        td = soup.find_all('td')
        table = str(td[0]).replace('<td bordercolor="#d0d7e5"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">', '').replace('</font></td>', '').split('<br/>')
        #tables.append(table)
        #text = td[1].get_text()
        return table
    except IndexError:
        return row

In [225]:
from time import time
start = time()
df['text'] = df.html.apply(parser)    
print((time() - start)/60)

3.0988337834676107


In [251]:
from time import time
start = time()
df['table'] = df.html.apply(parser)    
print((time() - start)/60)

3.2384648005167644


In [230]:
df.drop('NewCol', axis=1, inplace=True)

In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136494 entries, 0 to 136493
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   _id     136494 non-null  object
 1   url     136494 non-null  object
 2   html    136494 non-null  object
 3   time    136494 non-null  object
 4   text    136494 non-null  object
 5   table   136494 non-null  object
dtypes: object(6)
memory usage: 6.2+ MB


In [256]:
bad_index = df[(df['text'] == "")&(df['table'] == "")].index

In [323]:
temp = df[~df.index.isin(bad_index)]

In [324]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136457 entries, 0 to 136493
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   _id     136457 non-null  object
 1   url     136457 non-null  object
 2   html    136457 non-null  object
 3   time    136457 non-null  object
 4   text    136457 non-null  object
 5   table   136457 non-null  object
dtypes: object(6)
memory usage: 7.3+ MB


In [274]:
indices = np.where(temp['table'].apply(lambda x: len(x) < 6))

In [284]:
inds = indices[0]

In [289]:
temp.head()

,_id,url,html,time,text,table
0,{'$oid': '591382287c40e83fd74240bd'},http://www.nuforc.org/webreports/133/S133978.html,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\r\nxmlns:o=""urn:schemas-microsoft-com:office:office""\r\nxmlns:w=""urn:schemas-microsoft-com:office:word""\r\nxmlns:m=""http://schemas.microsoft.com/office/2004/12/omml""\r\nxmlns=""http://www.w3.org/TR/REC-html40"">\r\n\r\n<head>\r\n<meta http-equiv=Conten...",Wed May 10 21:12:07 2017,Light seen over mountain's east of Camp McGregor. It hovered in one spot. It looked like a helicopter light at first. But it was way to bright. Then it went to a higher height. Stayed there for a while. Then disappeared.((NUFORC Note: Report appears to us to be consistent with the sightin...,"[Occurred : 5/6/2017 05:00 (Entered as : 05/06/2017 05:00), Reported: 5/6/2017 4:10:01 AM 04:10, Posted: 5/6/2017, Location: Camp McGregor, NM, Shape: Light, Duration:10 minutes]"
1,{'$oid': '591389377c40e840c123dfef'},http://www.nuforc.org/webreports/133/S133977.html\n,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\r\nxmlns:o=""urn:schemas-microsoft-com:office:office""\r\nxmlns:w=""urn:schemas-microsoft-com:office:word""\r\nxmlns:m=""http://schemas.microsoft.com/office/2004/12/omml""\r\nxmlns=""http://www.w3.org/TR/REC-html40"">\r\n\r\n<head>\r\n<meta http-equiv=Conten...",Wed May 10 21:42:15 2017,"Light in sky stationary. Not a airplane or any known star. Flickering in brightness.((NUFORC Note: Possibly Venus. Source of the report elects to remain anonymous; provides no contact information. We could not find a city named ""Mojave"" in BC, Canada. PD))","[Occurred : 5/6/2017 04:50 (Entered as : 06/05/2017 04:50), Reported: 5/6/2017 5:00:54 AM 05:00, Posted: 5/6/2017, Location: Mojave (Canada), BC, Shape: Light, Duration:1:00]"
2,{'$oid': '591389397c40e840c123dff0'},http://www.nuforc.org/webreports/133/S133974.html\n,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\r\nxmlns:o=""urn:schemas-microsoft-com:office:office""\r\nxmlns:w=""urn:schemas-microsoft-com:office:word""\r\nxmlns:m=""http://schemas.microsoft.com/office/2004/12/omml""\r\nxmlns=""http://www.w3.org/TR/REC-html40"">\r\n\r\n<head>\r\n<meta http-equiv=Conten...",Wed May 10 21:42:17 2017,"Flying saucer descends, possibly lands in North Austin nature trailAt approximately 11:30 AM I was on the balcony of my apartment when I saw a flying object descending in the sky very rapidly. It traveled from west to east over Walnut Creek where it appeared to land on the trail. It definitely...","[Occurred : 5/5/2017 11:30 (Entered as : 05/05/17 11:30), Reported: 5/5/2017 12:18:44 PM 12:18, Posted: 5/6/2017, Location: Austin, TX, Shape: Disk, Duration:3 seconds]"
3,{'$oid': '5913893c7c40e840c123dff1'},http://www.nuforc.org/webreports/133/S133964.html\n,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\r\nxmlns:o=""urn:schemas-microsoft-com:office:office""\r\nxmlns:w=""urn:schemas-microsoft-com:office:word""\r\nxmlns:m=""http://schemas.microsoft.com/office/2004/12/omml""\r\nxmlns=""http://www.w3.org/TR/REC-html40"">\r\n\r\n<head>\r\n<meta http-equiv=Conten...",Wed May 10 21:42:20 2017,"While letting my dog out, a very bright white circle of light appeared suddenly in the sky. It seemed very low like small plane or helicopter height, but was brighter than anything I have seen before. It almost immediately began moving rapidly in a straight northeast direction, never diminishi...","[Occurred : 5/5/2017 03:00 (Entered as : 5/5/17 3:00), Reported: 5/5/2017 3:49:05 AM 03:49, Posted: 5/6/2017, Location: El Mirage, AZ, Shape: Circle, Duration:30 seconds]"
4,{'$oid': '5913893f7c40e840c123dff2'},http://www.nuforc.org/webreports/133/S133962.html\n,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\r\nxmlns:o=""urn:schemas-microsoft-com:office:office""\r\nxmlns:w=""urn:schemas-microsoft-com:office:word""\r\nxmlns:m=""http://schemas.microsoft.com/office/2004/12/omml""\r\nxmlns=""http://www.w3.org/TR/REC-html40"">\r\n\r\n<head>\r\n<meta http-equiv=Conten...",W

### Creating the "occurred" column

In [295]:
temp['occurred'] = temp['table'].apply(lambda x: x[0]).str.replace('<td bordercolor="#c0c0c0"><font color="#000000" face="Arial" style="FONT-SIZE:10pt">', '')

<ipython-input-295-169fa57615c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['occurred'] = temp['table'].apply(lambda x: x[0]).str.replace('<td bordercolor="#c0c0c0"><font color="#000000" face="Arial" style="FONT-SIZE:10pt">', '')


In [325]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136457 entries, 0 to 136493
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   _id     136457 non-null  object
 1   url     136457 non-null  object
 2   html    136457 non-null  object
 3   time    136457 non-null  object
 4   text    136457 non-null  object
 5   table   136457 non-null  object
dtypes: object(6)
memory usage: 7.3+ MB


### Create all extra columns in one go

In [326]:
from NLP_Case_study import table_parser

In [327]:
col_names = ['occurred', 'reported', 'posted', 'location', 'shape', 'duration']

In [329]:
for ind, name in enumerate(col_names):
    temp[name] = temp['table'].apply(table_parser, n=ind)

<ipython-input-329-75a7db7eef3e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[name] = temp['table'].apply(table_parser, n=ind)


In [347]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136457 entries, 0 to 136493
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   _id       136457 non-null  object
 1   url       136457 non-null  object
 2   html      136457 non-null  object
 3   time      136457 non-null  object
 4   text      136457 non-null  object
 5   table     136457 non-null  object
 6   occurred  136457 non-null  object
 7   reported  136457 non-null  object
 8   posted    136457 non-null  object
 9   location  136457 non-null  object
 10  shape     136457 non-null  object
 11  duration  136457 non-null  object
dtypes: object(12)
memory usage: 18.5+ MB


#### Save data as pickle file before you lose all your work!

In [338]:
temp.to_pickle('data/temp.pkl')

### Data Cleaning time

In [337]:
temp['reported'].apply(lambda x: x.replace('Reported:', '') if isinstance(x, str) else x)

0           5/6/2017 4:10:01 AM 04:10
1           5/6/2017 5:00:54 AM 05:00
2          5/5/2017 12:18:44 PM 12:18
3           5/5/2017 3:49:05 AM 03:49
4          5/4/2017 10:38:52 PM 22:38
                     ...             
136489                1/21/1998 19:06
136490                3/27/1997 18:08
136491                1/21/1998 18:22
136492                1/18/1998 10:05
136493                 2/1/1998 01:47
Name: reported, Length: 136457, dtype: object

In [345]:
temp[temp['reported'].apply(lambda x: isinstance(x, list))].index

Int64Index([  4306,   4308,   4309,   4310,   4312,   4314,   4315,   4316,
              4317,   4319,   4320,   4321,   4323,   4324,   4325,   4326,
              4327,   4328,   4330,   4331,   4333,   4334,   4335,   4336,
              4337,   4338,   4339,   4340,   4344,   4345,   7446,   7616,
             18334,  19578,  19628,  30306,  30308,  30309,  30310,  30312,
             30314,  30315,  30316,  30317,  30319,  30320,  30321,  30323,
             30324,  30325,  30326,  30327,  30328,  30330,  30331,  30333,
             30334,  30335,  30336,  30337,  30338,  30339,  30340,  30344,
             30345,  33446,  33616,  44334,  45578,  45628,  64640,  65320,
             65323,  82450,  82459,  92126, 106414, 111928, 111929, 113116,
            113121, 113154, 113188, 113212, 117309, 122463, 123645, 132345,
            133032],
           dtype='int64')

### Drop "Page Cannot be Found" entries (save 132345 and 117309)

In [348]:
temp.drop([4306,   4308,   4309,   4310,   4312,   4314,   4315,   4316,
              4317,   4319,   4320,   4321,   4323,   4324,   4325,   4326,
              4327,   4328,   4330,   4331,   4333,   4334,   4335,   4336,
              4337,   4338,   4339,   4340,   4344,   4345,   7446,   7616,
             18334,  19578,  19628,  30306,  30308,  30309,  30310,  30312,
             30314,  30315,  30316,  30317,  30319,  30320,  30321,  30323,
             30324,  30325,  30326,  30327,  30328,  30330,  30331,  30333,
             30334,  30335,  30336,  30337,  30338,  30339,  30340,  30344,
             30345,  33446,  33616,  44334,  45578,  45628,  64640,  65320,
             65323,  82450,  82459,  92126, 106414, 111928, 111929, 113116,
            113121, 113154, 113188, 113212, 122463, 123645, 133032], inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Fix anomalies

In [356]:
temp[temp['reported'].apply(lambda x: isinstance(x, list))]

,_id,url,html,time,text,table,occurred,reported,posted,location,shape,duration
117309,{'$oid': '591d3faab6747c5f388797a6'},http://www.nuforc.org/webreports/027/S27173.html\n,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Transitional//EN"">\r\n<HTML>\r\n<HEAD>\r\n <META HTTP-EQUIV=""Content-Type"" CONTENT=""text/html; charset=iso-8859-1"">\r\n <META NAME=""GENERATOR"" CONTENT=""Microsoft FrontPage 4.0"">\r\n</HEAD>\r\n<BODY>\r\n\r\n<CENTER><FONT FACE=""Arial,Helvetica"">&nbsp...",Thu May 18 06:31:06 2017,this is the third time i have seen these objects in less than a month in this area after the first time i was so scared that i wasn't sure if i had seen what i was sure i saw.....but since that first time i have been driven to try and explain what i have been seeing..so a friend and i went to th...,"[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 1/18/2003 02:30 (Entered as : 1-18-03 2:30 am), Reported: 1/20/2003 7:21:57 AM 07:21, Posted: 3/21/2003, Location: , Moundville, AL, Shape: Changing, Duration:45 min]","[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 1/18/2003 02:30 (Entered as : 1-18-03 2:30 am), Reported: 1/20/2003 7:21:57 AM 07:21, Posted: 3/21/2003, Location: , Moundville, AL, Shape: Changing, Duration:45 min]","[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 1/18/2003 02:30 (Entered as : 1-18-03 2:30 am), Reported: 1/20/2003 7:21:57 AM 07:21, Posted: 3/21/2003, Location: , Moundville, AL, Shape: Changing, Duration:45 min]","[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 1/18/2003 02:30 (Entered as : 1-18-03 2:30 am), Reported: 1/20/2003 7:21:57 AM 07:21, Posted: 3/21/2003, Location: , Moundville, AL, Shape: Changing, Duration:45 min]","[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 1/18/2003 02:30 (Entered as : 1-18-03 2:30 am), Reported: 1/20/2003 7:21:57 AM 07:21, Posted: 3/21/2003, Location: , Moundville, AL, Shape: Changing, Duration:45 min]","[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 1/18/2003 02:30 (Entered as : 1-18-03 2:30 am), Reported: 1/20/2003 7:21:57 AM 07:21, Posted: 3/21/2003, Location: , Moundville, AL, Shape: Changing, Duration:45 min]","[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 1/18/2003 02:30 (Entered as : 1-18-03 2:30 am), Reported: 1/20/2003 7:21:57 AM 07:21, Posted: 3/21/2003, Location: , Moundville, AL, Shape: Changing, Duration:45 min]"
132345,{'$oid': '591dc5ecb6747c5f3887d262'},http://www.nuforc.org/webreports/010/S10139.html\n,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Transitional//EN"">\r\n<HTML>\r\n<HEAD>\r\n <META HTTP-EQUIV=""Content-Type"" CONTENT=""text/html; charset=iso-8859-1"">\r\n <META NAME=""GENERATOR"" CONTENT=""Microsoft FrontPage 4.0"">\r\n</HEAD>\r\n<BODY>\r\n\r\n<CENTER><FONT FACE=""Arial,Helvetica"">&nbsp...",Thu May 18 16:03:56 2017,"seen in the distance, lights that went around the craft in a strob,circeler around it. was hovering for about 3 min's and went stright up and disaperd.seen in the distance, lights that went around the craft in a strob,circeler around it. was hovering for about 3 min's and went stright up and dis...","[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 10/6/1999 22:46 (Entered as : 10/06/99 22:46), Reported: 10/7/1999 04:36, Posted: 3/21/2003, Location: May's Landing, Mays Landing, NJ, Shape: Disk, Duration:3 min]","[<td bordercolor=""#c0c0c0""><font color=""#000000"" face=""Arial"" style=""FONT-SIZE:10pt"">Occurred : 10/6/1999 22:46 (Entered as : 10/06/99 22:46), Reported: 10/7/1999 04:36, Posted: 3/21/2003, Location: May's Landing, Mays Landing, NJ, Shape: Disk, Duration:3 min]","[<td 

In [357]:
anomalies = temp[temp['reported'].apply(lambda x: isinstance(x, list))]

In [373]:
temp.loc[117309,'table'] = ['Occurred : 1/18/2003 02:30  (Entered as : 1-18-03 2:30 am)', 'Reported: 1/20/2003 7:21:57 AM 07:21', 'Posted: 3/21/2003','Location: Moundville, AL', 'Shape: Changing', 'Duration:45 min']

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-373-4bea86ae3b08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[117309,'table'] = ['Occurred : 1/18/2003 02:30  (Entered as : 1-18-03 2:30 am)', 'Reported: 1/20/2003 7:21:57 AM 07:21', 'Posted: 3/21/2003','Location: Moundville, AL', 'Shape: Changing', 'Duration:45 min']


In [374]:
temp.loc[117309, 'table']

['Occurred : 1/18/2003 02:30  (Entered as : 1-18-03 2:30 am)',
 'Reported: 1/20/2003 7:21:57 AM 07:21',
 'Posted: 3/21/2003',
 'Location: Moundville, AL',
 'Shape: Changing',
 'Duration:45 min']

In [376]:
temp.loc[132345, 'table'] = ['Occurred : 10/6/1999 22:46  (Entered as : 10/06/99 22:46)', 'Reported: 10/7/1999 04:36', 'Posted: 3/21/2003', "Location: May's Landing, NJ", 'Shape: Disk', 'Duration:3 min']

<ipython-input-376-939a2431caf3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[132345, 'table'] = ['Occurred : 10/6/1999 22:46  (Entered as : 10/06/99 22:46)', 'Reported: 10/7/1999 04:36', 'Posted: 3/21/2003', "Location: May's Landing, NJ", 'Shape: Disk', 'Duration:3 min']


In [377]:
temp.loc[132345, 'table']

['Occurred : 10/6/1999 22:46  (Entered as : 10/06/99 22:46)',
 'Reported: 10/7/1999 04:36',
 'Posted: 3/21/2003',
 "Location: May's Landing, NJ",
 'Shape: Disk',
 'Duration:3 min']

In [386]:
for ind, name in enumerate(col_names):
    temp.loc[132345, name] = temp.loc[132345,'table'][ind]

<ipython-input-386-019bdb318971>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[132345, name] = temp.loc[132345,'table'][ind]


### One last check to ensure we are clean before moving on 

In [390]:
for name in col_names:
    print(temp[temp[name].apply(lambda x: isinstance(x, list))].index)

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')


### Clean!!!

### Data Cleaning time
- remove "Reported"
- remove "Posted" 
- remove "Location"
- remove "Shape"
- remove "Duration"

In [400]:
from NLP_Case_study import removal
cleaned = temp.copy()

In [ ]:
for name in col_names:
    temp[name].apply(removal, name=name)

In [401]:
cleaned['occurred'] = cleaned['occurred'].apply(removal, name=test)

### Remove indices where the table column entry is greater than six which indicates bad html

In [416]:
bad_table_index = temp['table'][temp.table.apply(lambda x: len(x) > 6)].index

In [417]:
cleaned.drop(bad_table_index, inplace=True)

In [419]:
cleaned['table'][cleaned.table.apply(lambda x: len(x) > 6)]

Series([], Name: table, dtype: object)

# Make sure to come back to this ugliness!

In [425]:
cleaned['occurred'][cleaned.occurred.apply(lambda x: len(x) > 50)]

2927                                      10/11/2016 13:35  (Entered as : 10-11-2016 1:35 PM)
3238                                      9/26/2016 07:30  (Entered as : 09/26/2016 07:30 AM)
5091                 6/27/2016 03:46  (Entered as : 27th June 2016 03.46 AM Local Jakarta Ti)
7047                                      2/11/2016 06:45  (Entered as : 02/11/2016 7:40 a.m)
7051                                     2/10/2016 19:30  (Entered as : 02/9&amp;10/16 19:30)
7772                                      12/30/2015 19:30  (Entered as : 12/30/2015 7:30 pm)
8150                                      12/5/2015 10:56  (Entered as : December 0 10:56 am)
15702                                     11/27/2014 05:45  (Entered as : 11/27/2014 5:45 PM)
17499                                     9/18/2014 02:08  (Entered as : 18/09/2014 02:08 AM)
19227                                     7/19/2014 21:40  (Entered as : 07/19/2014 2140 hrs)
21568                                     4/29/2014 22:30  (

In [436]:
cleaned['reported'] = cleaned['reported'].apply(removal, name='Reported')

In [450]:
cleaned['posted'] = cleaned['posted'].apply(removal, name='Posted')

In [458]:
cleaned['location'] = cleaned['location'].apply(removal, name='Location')

In [462]:
cleaned['duration'] = cleaned['duration'].apply(removal, name='Duration')

In [465]:
cleaned['shape'] = cleaned['shape'].apply(removal, name='Shape')

In [485]:
def ufunc(row):
    if len(row) == 0:
        return 'Unknown'
    elif row == 'TRIANGULAR':
        return 'Triangle'
    else: 
        return row

In [486]:
cleaned['shape'] = cleaned['shape'].apply(ufunc)

In [483]:
cleaned['shape'] = cleaned['shape'].apply(lambda x: x[0].upper() + x[1:])

In [487]:
cleaned['shape'].value_counts()

Light        27898
Circle       14613
Unknown      13519
Triangle     12662
Fireball     10972
Other         9219
Sphere        9075
Disk          7982
Oval          5929
Formation     4341
Changing      3266
Cigar         3115
Flash         2442
Rectangle     2278
Cylinder      2059
Diamond       1986
Chevron       1584
Teardrop      1208
Egg           1168
Cone           568
Cross          431
Delta            8
Round            2
Crescent         2
Flare            1
Changed          1
Dome             1
Hexagon          1
Pyramid          1
Name: shape, dtype: int64

In [489]:
cleaned.to_pickle('data/cleaned.pkl')

### Saving as Feather file

In [5]:
dropped.reset_index(drop=True, inplace=True)

In [6]:
dropped.to_feather('data/cleaned.feather')

In [2]:
dropped.to_csv('data/cleaned.csv')

In [496]:
dropped.drop(['url', 'html'], axis=1, inplace=True)

In [497]:
dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136332 entries, 0 to 136493
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   _id       136332 non-null  object
 1   time      136332 non-null  object
 2   text      136332 non-null  object
 3   table     136332 non-null  object
 4   occurred  136332 non-null  object
 5   reported  136332 non-null  object
 6   posted    136332 non-null  object
 7   location  136332 non-null  object
 8   shape     136332 non-null  object
 9   duration  136332 non-null  object
dtypes: object(10)
memory usage: 16.4+ MB


In [506]:
pd.set_option('display.max_rows', 1400)

In [508]:
#dropped['duration'][dropped['duration'].str.contains(' to ')].str.replace(' to ', '-')

In [514]:
#dropped['duration'][dropped['duration'].str.contains('minute')].str.replace('+','').str.replace('~', '').str.replace('&gt;', '').str.replace('&lt;','').head(500)

### Bag of Words approach to time series

In [521]:
def duration_cleaning(row):
    if len(row) == 0:
        return "Unknown"
    else: 
        return row.replace('+','').replace('~', '').replace('&gt;', '').replace('&lt;','')

In [523]:
dropped['duration'] = dropped['duration'].apply(duration_cleaning)

In [526]:
corpus = dropped['duration'].values.tolist()

In [625]:
words = []
for phrase in corpus:
    text = phrase.split()
    for word in text:
        if word in ['minute', 'minutes', 'mimutes', 'mimute', 'minutess', 'minutes?', 'mintues', 'minuts', 'minuets', 'mints', 'min','mins', 'min.',
                    'Min.', 'Mins', 'Mins.', 'mins.', 'Min', 'Minutes', 'Minute', 'mins.', 'Min', 'Minutes', 'Minute', 'minutes,', 'minutes.', 'minuites', 'minites', 'min?',
                    'MINUTES', 'SECONDS', 'MIN', 'MIN.', 'MINS', 'MINUTE', 'HOURS', 'HOUR', 'HRS.', 'SEC', 'SEC.', 'SECS', 'HRS',
                    'second', 'seconds', 'sec', 'sec.', 'secs', 'secs.', 'Sec.', 'Sec', 'seconds?', 'Seconds', 'Second', 'secounds', 'seconds.',
                    'hour', 'hours', 'Hour', 'Hours', 'hr', 'hrs.', 'hrs', 'hours?',
                    'ongoing', 'Ongoing', 'day', 'days', 'unknown', 'Unknown', 'UNKNOWN', 'to']:
            pass
        elif word.isdigit():
            pass
        else: words.append(word)
len(words)

41916

In [626]:
new_words = []
for word in words:
    word = word.replace(':', '').replace('-', '')
    if len(word) < 1:
        pass
    else: 
        if word.isdigit() or word.isdecimal():
            pass
        elif word[0].isdigit():
            pass
        else:
            new_words.append(word)
len(new_words)

18909

In [624]:
from collections import Counter
test = Counter(new_words)
test.most_common(150)

[('about', 772),
 ('few', 689),
 ('a', 655),
 ('or', 581),
 ('one', 389),
 ('approx', 386),
 ('less', 364),
 ('than', 346),
 ('?', 322),
 ('approx.', 277),
 ('still', 267),
 ('several', 245),
 ('night', 226),
 ('Few', 224),
 ('and', 216),
 ('About', 203),
 ('going', 192),
 ('half', 187),
 ('on', 181),
 ('an', 168),
 ('so', 161),
 ('two', 161),
 ('five', 159),
 ('more', 153),
 ('at', 151),
 ('sure', 145),
 ('A', 141),
 ('over', 136),
 ('not', 131),
 ('least', 125),
 ('of', 122),
 ('ten', 118),
 ('Still', 112),
 ('plus', 110),
 ('all', 108),
 ('Several', 103),
 ('the', 95),
 ('there', 93),
 ('Less', 86),
 ('three', 84),
 ('Approx.', 80),
 ('in', 79),
 ('One', 77),
 ('aprox', 76),
 ('for', 75),
 ('Not', 73),
 ('time', 72),
 ('know', 70),
 ('couple', 69),
 ('each', 66),
 ('Approx', 66),
 ('All', 64),
 ('happening', 63),
 ('nights', 62),
 ('/', 62),
 ('I', 61),
 ('hr.', 61),
 ('around', 60),
 ('then', 52),
 ('unsure', 49),
 ('approximately', 49),
 ('maybe', 47),
 ("don't", 45),
 ('continuou

In [10]:
dropped.drop(['table'], axis=1, inplace=True)
dropped.head()

,_id,time,text,occurred,reported,posted,location,shape,duration
0,{'$oid': '591382287c40e83fd74240bd'},Wed May 10 21:12:07 2017,Light seen over mountain's east of Camp McGregor. It hovered in one spot. It looked like a helicopter light at first. But it was way to bright. Then it went to a higher height. Stayed there for a while. Then disappeared.((NUFORC Note: Report appears to us to be consistent with the sightin...,5/6/2017 05:00 (Entered as : 05/06/2017 05:00),5/6/2017 4:10:01 AM 04:10,5/6/2017,"Camp McGregor, NM",Light,10 minutes
1,{'$oid': '591389377c40e840c123dfef'},Wed May 10 21:42:15 2017,"Light in sky stationary. Not a airplane or any known star. Flickering in brightness.((NUFORC Note: Possibly Venus. Source of the report elects to remain anonymous; provides no contact information. We could not find a city named ""Mojave"" in BC, Canada. PD))",5/6/2017 04:50 (Entered as : 06/05/2017 04:50),5/6/2017 5:00:54 AM 05:00,5/6/2017,"Mojave (Canada), BC",Light,1:00
2,{'$oid': '591389397c40e840c123dff0'},Wed May 10 21:42:17 2017,"Flying saucer descends, possibly lands in North Austin nature trailAt approximately 11:30 AM I was on the balcony of my apartment when I saw a flying object descending in the sky very rapidly. It traveled from west to east over Walnut Creek where it appeared to land on the trail. It definitely...",5/5/2017 11:30 (Entered as : 05/05/17 11:30),5/5/2017 12:18:44 PM 12:18,5/6/2017,"Austin, TX",Disk,3 seconds
3,{'$oid': '5913893c7c40e840c123dff1'},Wed May 10 21:42:20 2017,"While letting my dog out, a very bright white circle of light appeared suddenly in the sky. It seemed very low like small plane or helicopter height, but was brighter than anything I have seen before. It almost immediately began moving rapidly in a straight northeast direction, never diminishi...",5/5/2017 03:00 (Entered as : 5/5/17 3:00),5/5/2017 3:49:05 AM 03:49,5/6/2017,"El Mirage, AZ",Circle,30 seconds
4,{'$oid': '5913893f7c40e840c123dff2'},Wed May 10 21:42:23 2017,"A fire ball was moving in the atmosphere while stopping, going, and even changing directions.11:17 p.m CST. (The before situation) This is when I first snuck outside to go stargazing for a while. When I first came outside I began to watch the stars and relax as the last day of school for me is t...",5/4/2017 23:34 (Entered as : 5/4/17 23:34),5/4/2017 10:38:52 PM 22:38,5/6/2017,"York, NE",Fireball,0:18
